In [8]:
# ============================================
# Paso 0 - Limpieza total de vectordb
# ============================================
import chromadb
import shutil
import os

VDB_PATH = "./vectordb"
COLLECTION_NAME = "langchain"

In [9]:
# Opción 1: Eliminar carpeta físicamente
'''
if os.path.exists(VDB_PATH):
    shutil.rmtree(VDB_PATH)
    print("🗑️ Carpeta ./vectordb eliminada por completo.")
else:
    print("ℹ️ No existe carpeta ./vectordb, nada que limpiar.")

'''

'\nif os.path.exists(VDB_PATH):\n    shutil.rmtree(VDB_PATH)\n    print("🗑️ Carpeta ./vectordb eliminada por completo.")\nelse:\n    print("ℹ️ No existe carpeta ./vectordb, nada que limpiar.")\n\n'

In [10]:
# Opción 2: Eliminar colección desde Chroma (por si persiste algún registro)
'''
client = chromadb.PersistentClient(path=VDB_PATH)
try:
    client.delete_collection(COLLECTION_NAME)
    print(f"🗑️ Colección '{COLLECTION_NAME}' eliminada.")
except Exception:
    print(f"ℹ️ No había colección '{COLLECTION_NAME}' para eliminar.")
'''

'\nclient = chromadb.PersistentClient(path=VDB_PATH)\ntry:\n    client.delete_collection(COLLECTION_NAME)\n    print(f"🗑️ Colección \'{COLLECTION_NAME}\' eliminada.")\nexcept Exception:\n    print(f"ℹ️ No había colección \'{COLLECTION_NAME}\' para eliminar.")\n'

In [ ]:
# ============================================
# Paso 1 - Configuración API y librerías
# ============================================
import os
import re
import pdfplumber
import chromadb
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain.schema import Document

from dotenv import load_dotenv
import os

# Cargar variables del archivo .env
load_dotenv()

# Leer la API key desde el entorno
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")



# Cliente persistente de Chroma
persistent_client = chromadb.PersistentClient(path=VDB_PATH)


In [12]:
# ============================================
# Paso 2 - Función de limpieza de texto
# ============================================
def clean_text(text: str) -> str:
    patrones = [
        r"FORMATO REQUISICIÓN.*",
        r"ECOPETROL DESARROLLO DE PROYECTO.*",
        r"Todos los derechos reservados.*",
        r"___________________________________________________________________",
        r"EDP-F-046.*Versión.*",
        r"MR.*#:.*CAS.*",
        r"Página\s+\d+\s+de\s+\d+",
    ]
    for p in patrones:
        text = re.sub(p, "", text, flags=re.IGNORECASE)
    text = re.sub(r"\s{2,}", " ", text)  # normalizar espacios
    return text.strip()


In [13]:
# ============================================
# Paso 3 - Extractor de PDF (texto + tablas)
# ============================================
def extract_pdf_content(filepath: str) -> List[Document]:
    docs = []
    with pdfplumber.open(filepath) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            # --- Extraer texto limpio ---
            text = page.extract_text() or ""
            text = clean_text(text)

            if text:
                docs.append(
                    Document(
                        page_content=text,
                        metadata={"source": filepath, "page": i, "type": "text"}
                    )
                )

            # --- Extraer tablas ---
            try:
                tables = page.extract_tables()
                for t in tables:
                    if not t:
                        continue
                    # Convertir filas en texto legible
                    table_text = "\n".join(
                        [" | ".join(filter(None, row)) for row in t if any(row)]
                    )
                    table_text = clean_text(table_text)
                    if table_text:
                        docs.append(
                            Document(
                                page_content=table_text,
                                metadata={"source": filepath, "page": i, "type": "table"}
                            )
                        )
            except Exception as e:
                print(f"⚠️ No se pudo procesar tabla en página {i}: {e}")

    return docs



In [14]:
# ============================================
# Paso 4 - División en chunks y guardado en vectordb
# ============================================
def add_files_to_vectordb(filepath: str):
    # Eliminar colección previa para evitar duplicados
    try:
        persistent_client.delete_collection(COLLECTION_NAME)
        print(f"🗑️ Colección '{COLLECTION_NAME}' eliminada.")
    except Exception:
        print(f"ℹ️ No había colección previa '{COLLECTION_NAME}'.")

    docs = extract_pdf_content(filepath)

    # Dividir en fragmentos manejables
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=100,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    splits = text_splitter.split_documents(docs)

    # Guardar en Chroma en colección limpia
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=OpenAIEmbeddings(),
        persist_directory=VDB_PATH,
        collection_name=COLLECTION_NAME
    )

    print(f"✅ {len(splits)} fragmentos indexados en ./vectordb (colección {COLLECTION_NAME})")
    return vectorstore


In [15]:
# ============================================
# Paso 5 - Ejemplo de uso
# ============================================
if __name__ == "__main__":
    vectorstore = add_files_to_vectordb(
        "./data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf"
    )


🗑️ Colección 'langchain' eliminada.
✅ 545 fragmentos indexados en ./vectordb (colección langchain)


In [16]:
# ============================================
# Paso 6 - Verificar qué documentos hay en vectordb
# ============================================
def get_unique_sources_list(chroma_settings):
    collection_data = chroma_settings.get_collection('langchain').get(
        include=['embeddings', 'documents', 'metadatas']
    )
    metadatas = collection_data['metadatas']

    sources = set()
    for metadata in metadatas:
        source = metadata.get('source', None)
        if source:
            sources.add(source)

    # Obtener solo el nombre de archivo de cada ruta
    file_names = list(set(source.split('/')[-1] for source in sources))
    return file_names

print("📂 Documentos en vectordb:", get_unique_sources_list(persistent_client))


📂 Documentos en vectordb: ['CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf']


In [19]:
retriever = vectorstore.as_retriever(search_kwargs={"k":8})
results = retriever.get_relevant_documents("¿Cuál es el alcance del suministro?")
for r in results:
    print("\n--- CHUNK ---")
    print(r.page_content[:500])
    print("Metadatos:", r.metadata)



--- CHUNK ---
2.0 ALCANCE
Este documento describe el alcance del suministro y servicios del PROVEEDOR, sus
responsabilidades, plan de ejecución y demás aspectos generales para el diseño; fabricación;
suministro; asistencia para el montaje y pruebas del PAQUETE de inyección de químicos de Polímero
Aniónico AX-75317 y los Quill´s de Inyección Retráctil roscado, completamente operacionales y
funcionales.
Los detalles relevantes a los requerimientos de proceso, características constructivas, garantías de
desempeñ
Metadatos: {'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'page': 58, 'type': 'text'}

--- CHUNK ---
SECCIÓN | DESCRIPCIÓN
2 | ALCANCE DEL SUMINISTRO
10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)
Metadatos: {'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'page': 1}

--- CHUNK ---
1.2. GLOSARIO .............................................................................................

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k":3})
results = retriever.get_relevant_documents("¿Cuales son los tags y cantides de los sistemas de inyeccion solcitiados?")
for r in results:
    print("\n--- CHUNK ---")
    print(r.page_content[:500])
    print("Metadatos:", r.metadata)


--- CHUNK ---
Descripción | Código de
Catálogo
ECOPETROL
Ítem | Cantidad | Unidad | TAG
El paquete contara con un sistema de control PLC dedicado
a gestionar todas las funciones del paquete, sus cajas
Potencia, Instrumentación, Instrumentos y Control NEMA
4X.
Las bombas dosificaran químico aniónico hacia el patín de
almacenamiento Tanques de Decantación ATR-7551
A/B/C/D/E/F Cabezal de 8” PWA-751407-AA1A33-01-N-
0.0”
2 | 1 | UND | AX-
75317 | Suministro de un (1) quill de inyección retráctil (boquilla de
inyec
Metadatos: {'page': 33, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'table'}

--- CHUNK ---
PAQUETE DE INYECCIÓN DE POLIMERO ANIÓNICO AX-75317
TIPO | SISTEMA AUTOMÁTICO PARA PREPARACIÓN DE POLÍMERO EN POLVO
BOMBAS DOSIFICADORAS CON MOTOR ELÉCTRICO
CAPACIDAD | 12-18 GPH
PRESIÓN DE DESCARGA | 100 PSIG
POTENCIA DE MOTOR BOMBA | 1 HP
QUILLS DE INYECCIÓN | 1
CAPACIDAD TANQUE | 4000 L
POTENCIA DE MOTOR AGITADORES | 1 HP
Metadatos: {'source': '

In [18]:
# Ver todos los chunks de la página 32 (texto + tablas)
collection = persistent_client.get_collection("langchain")
docs_data = collection.get(include=["documents", "metadatas"])

docs_p32 = [
    (doc, meta) for doc, meta in zip(docs_data["documents"], docs_data["metadatas"])
    if meta.get("page") == 32
]

print(f"🔎 Total de chunks en página 32: {len(docs_p32)}\n")

for doc, meta in docs_p32[:10]:  # muestra hasta 10
    print("\n--- CHUNK ---")
    print(doc[:500])  # primeras 500 chars
    print("Metadatos:", meta)


🔎 Total de chunks en página 32: 8


--- CHUNK ---
MAYORES
PAQUETES DE INYECCIÓN DE QUÍMICOS VICEPRESIDENCIA DE INGENIERÍA Y PROYECTOS
CÓDIGO Elaborado
Versión: 1
EDP-F-046 20/11/2017
MR #:
INGENIERÍA DETALLADA ESTACIONES MÓDULO
CAS-09991-MER-MR- REV: 4
INTEGRAL
000003
Documento:
Requisición de Materiales FECHA:
Especialidad Mecánica
Paquete de inyección de 14/11/2024
químicos
Todos los equipos componentes del paquete de inyección de químicos, así como los componentes de los
quill´s de inyección retráctiles deben ser diseñados y seleccionados pa
Metadatos: {'type': 'text', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'page': 32}

--- CHUNK ---
válida por doce (12) meses, contados a partir de la fecha de puesta en funcionamiento del equipo y sus
componentes, o dieciocho (18) meses a partir de la entrega a satisfacción de estos por parte de
ECOPETROL S.A. Durante este período la garantía debe cubrir la reparación o reemplazo de los
componentes defectuosos